In [1]:
import pandas as pd
from pandas import Series, DataFrame
from pandarallel import pandarallel
pandarallel.initialize(progress_bar=True)
import numpy as np
folder = "/data/PDGA_MAP4/"
from Levenshtein import distance as lev_dist

INFO: Pandarallel will run on 8 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.


# Filtering

In [2]:
df_all = pd.read_pickle(folder+"pickles/all_sequences_with_NN_prop_helicity-hem_correct_dist.pkl")
pdga = df_all.query("Set != 'training' and Set!= 'test' and isPredActive == True and isPredNotHemolytic == True")

In [3]:
df_all = df_all.drop_duplicates("Sequence").copy()

In [5]:
len(df_all.query("isPredActive == True and isPredNotHemolytic == True"))

6300

In [16]:
len(df_all)

715658

In [19]:
pdga_filtered = pdga.query("fraction_PredHelical > 0.8 and HydroMoment > 0.3 and (dist_Training > 4 and dist_Training <= 7) and dist_Test > 4 and length <= 15 and D_AA == False").reset_index(drop=True)

In [20]:
len(pdga_filtered)

152

In [21]:
cols = ['ID', 'Sequence', 'prediction', 'prediction_hem', 'length', 
       'dist_Training', 'NN_Training', 'dist_Test', 'NN_Test', 'dist_Training_', 'NN_Training_', 'dist_Test_', 'NN_Test_', 'map_dist_Training',
       'map_NN_Training', 'map_dist_Test', 'map_NN_Test', 'fraction_PredHelical', 'HydroMoment', 'SMILES', 'MAP4', 'JD']

In [22]:
pdga_filtered[cols].to_csv(folder+"data/pdga_filtered.csv", index=False)
pdga_filtered[cols].to_csv("data/pdga_filtered.csv", index=False)

# Clustering

In [23]:
from rdkit.ML.Cluster.Butina import ClusterData
clusters_pdga = ClusterData(pdga_filtered.Sequence.to_list(), len(pdga_filtered), 5, distFunc=lev_dist)

In [24]:
len(clusters_pdga)

22

In [25]:
for cluster in clusters_pdga:
    for idx in cluster:
        if pdga_filtered.iloc[idx].Sequence == "ANWKKILKRLCDI":
            idx_clusters = cluster
for idx in idx_clusters:
    print(pdga_filtered.iloc[idx].Sequence)

ANWKKILKRLCDI


In [26]:
pdga_filtered.iloc[list(idx_clusters)][cols].to_csv(folder+"data/pdga_same_cluster.csv", index=False)
pdga_filtered.iloc[list(idx_clusters)][cols].to_csv("data/pdga_same_cluster.csv", index=False)

In [38]:
idx_pdga = []
for cluster_pdga in clusters_pdga:
    idx_pdga.append(cluster_pdga[0])

In [39]:
idx_pdga

[7,
 98,
 38,
 139,
 12,
 33,
 130,
 119,
 53,
 102,
 0,
 114,
 89,
 128,
 143,
 58,
 81,
 4,
 79,
 32,
 10,
 125]

In [40]:
pdga_filtered.iloc[idx_pdga][cols].to_csv(folder+"data/pdga_filtered_clustered.csv", index=False)
pdga_filtered.iloc[idx_pdga][cols].to_csv("data/pdga_filtered_clustered.csv", index=False)